In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Data prep
shot_df = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df = shot_df.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df = shot_df[shot_df['SEASON'] != "GAME_ID"]

#convert columns to numbers
shot_df[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]] = shot_df[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]].apply(pd.to_numeric)

#convert missed/made to 0/1
shot_df['SHOT_RESULT'] = shot_df['SHOT_RESULT'].replace({'missed': 0})
shot_df['SHOT_RESULT'] = shot_df['SHOT_RESULT'].replace({'made': 1})

#classify shot distances
shot_df.loc[shot_df['PT_VALUE'] == 3, 'SHOT_TYPE'] = "Three Pointer"
shot_df.loc[(shot_df['SHOT_DIST'] >= 15) & (shot_df['PT_VALUE'] == 2), 'SHOT_TYPE'] = "Long Two-Pointer"
shot_df.loc[(shot_df['SHOT_DIST'] >= 5) & (shot_df['SHOT_DIST'] < 15), 'SHOT_TYPE'] = "Midrange"
shot_df.loc[shot_df['SHOT_DIST']< 5, 'SHOT_TYPE'] = "Layup"

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Catogorical Features
shot_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222317 entries, 1 to 222317
Data columns (total 22 columns):
SEASON                    222317 non-null int64
GAME_ID                   222317 non-null int64
SV_GAME_ID                222317 non-null int64
GAME_DATE                 222317 non-null object
TEAM_ID                   222317 non-null int64
SV_TEAM_ID                222317 non-null int64
SHOT_TAKER_TEAM           222317 non-null object
PERSON_ID                 222317 non-null int64
SV_PLAYER_ID              222317 non-null int64
SHOT_RESULT               222317 non-null int64
PERIOD                    222317 non-null int64
GAME_CLOCK                222317 non-null int64
WALL_CLOCK                222317 non-null int64
DRIBBLES                  222317 non-null int64
SHOT_DIST                 222317 non-null float64
TOUCH_TIME                222317 non-null float64
CLOSE_DEF_PERSON_ID       222317 non-null int64
CLOSE_DEF_SV_PLAYER_ID    222317 non-null int64
CLOSE_DEF_DIST          

In [30]:
# remove season, game_ID, SV_game_ID, Wall_clock, PT-Value, PTS
#shot_df.drop(['SEASON','GAME_ID','SV_GAME_ID','GAME_DATE', 'WALL_CLOCK', 'SHOT_TAKER_TEAM','PT_VALUE', 'PTS', 'SHOT_TYPE'],axis=1,inplace=True)


#drop everything but shot distance
shot_df = shot_df[['SHOT_RESULT', "SHOT_DIST", "DRIBBLES", "CLOSE_DEF_DIST", "TOUCH_TIME", "SV_PLAYER_ID", "CLOSE_DEF_SV_PLAYER_ID"]]
#shot_df = shot_df[['SHOT_RESULT', "SHOT_DIST", "DRIBBLES", "CLOSE_DEF_DIST", "TOUCH_TIME"]]

shot_df.head()

,SHOT_RESULT,SHOT_DIST,DRIBBLES,CLOSE_DEF_DIST,TOUCH_TIME
1,0,40.10,3,0.68,1.87
2,0,2.43,16,2.90,16.96
3,1,21.09,0,4.03,0.84
4,1,4.60,13,0.97,9.88
5,0,22.59,3,4.52,5.42


In [31]:
from sklearn.model_selection import train_test_split


In [32]:
X_train, X_test, y_train, y_test = train_test_split(shot_df.drop('SHOT_RESULT',axis=1), 
                                                    shot_df['SHOT_RESULT'], test_size=0.3, 
                                                    random_state=101)

In [33]:
from sklearn.linear_model import LogisticRegression


In [34]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict_proba(X_test)
shot_probability = []
for i in predictions:
    shot_probability.append(round(i[1], 4))


In [35]:
X_test['SHOT_PROBABILITY'] = shot_probability

In [10]:
#who takes the best shots??

In [11]:
#eligible_shooters = X_test.groupby('SV_PLAYER_ID')
#eligible_shooters =  eligible_shooters.filter(lambda x: len(x) > 150)  # pandas 0.13.1

eligible_shooters_all = shot_df[['SV_PLAYER_ID']].groupby('SV_PLAYER_ID')
eligible_shooters_all = eligible_shooters_all.filter(lambda x: len(x) > 500)  # pandas 0.13.1
eligible_shooters_all = eligible_shooters_all[['SV_PLAYER_ID']].drop_duplicates()

eligible_shooters = X_test.groupby('SV_PLAYER_ID')
eligible_shooters = eligible_shooters.filter(lambda x: len(x) > 5)  # pandas 0.13.1

eligible_shooters =  pd.merge(eligible_shooters_all, eligible_shooters,  how='left', left_on='SV_PLAYER_ID', right_on = 'SV_PLAYER_ID')



In [12]:
len(eligible_shooters_all)

187

In [13]:
best_shooter = (eligible_shooters.groupby(['SV_PLAYER_ID'], as_index=False).mean()
            .groupby('SV_PLAYER_ID')['SHOT_PROBABILITY'].mean())

In [14]:
best_shooter.sort_values(ascending = False)


SV_PLAYER_ID
4497    0.590481
5163    0.569746
3818    0.566666
4884    0.566125
4904    0.550669
4886    0.546609
4899    0.545131
3983    0.544481
4764    0.543641
5015    0.540607
5157    0.535093
3842    0.532508
5192    0.530340
4722    0.529263
4477    0.528603
4631    0.526256
4290    0.519785
5156    0.518755
5352    0.517170
5262    0.515778
5191    0.511560
4718    0.510317
5318    0.504434
5350    0.504352
5434    0.501821
5293    0.499527
5185    0.499386
4613    0.499271
5021    0.498915
4720    0.496578
          ...   
4694    0.401688
4152    0.401236
4244    0.400469
4487    0.400281
4890    0.399412
4725    0.399399
5073    0.398530
4563    0.397669
4905    0.397445
4660    0.395013
4139    0.394972
5433    0.393112
5160    0.392250
5334    0.388644
3930    0.388439
4612    0.385307
4633    0.384446
3835    0.384259
4296    0.383471
5012    0.383348
3407    0.382040
3971    0.381224
5095    0.380222
4889    0.379375
5253    0.378185
5014    0.376505
5204    0.373135
3

In [15]:
# link name and NBA player ID
text = open("Hackathon_player_names_matched_team.txt")
line = text.readline()
line = text.readline()
IDtoName = {} # ID as keys
NametoID = {} # name as keys

while line != "" :
    data = line.split()
    name = data[4] + " " + data[5]
    name = name.replace("\"","")
    ID = (float(data[3]))
    if ID not in IDtoName:
        IDtoName[ID] = name
    if name not in NametoID:
        NametoID[name] = ID
    line = text.readline()
    
#print(NametoID["LeBron James"])
#print(IDtoName[2544])



shot_df_name = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df_name = shot_df_name.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df_name = shot_df_name[shot_df_name['SEASON'] != "GAME_ID"]

shot_df_name = shot_df_name[['PERSON_ID', 'SV_PLAYER_ID']]

#convert columns to numbers
shot_df_name[['PERSON_ID', 'SV_PLAYER_ID']] = shot_df_name[['PERSON_ID', 'SV_PLAYER_ID']].apply(pd.to_numeric).drop_duplicates()


names_df = pd.DataFrame(list(NametoID.items()), columns = ['NAME', 'PERSON_ID'])

names_df = pd.merge(names_df, shot_df_name,  how='left', left_on='PERSON_ID', right_on = 'PERSON_ID')

names_df = names_df.dropna()




/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
best_shooter_df = pd.DataFrame({'SV_PLAYER_ID':best_shooter.index, 'AVG SHOT PROB':best_shooter.values})
best_shooter_df = pd.merge(best_shooter_df, names_df,  how='left', left_on='SV_PLAYER_ID', right_on = 'SV_PLAYER_ID')



In [17]:
#shot_df.drop(['SEASON','GAME_ID','SV_GAME_ID','GAME_DATE', 'WALL_CLOCK', 'SHOT_TAKER_TEAM','PT_VALUE', 'PTS', 'SHOT_TYPE'],axis=1,inplace=True)
best_shooter_df.drop(['SV_PLAYER_ID', 'PERSON_ID'], axis = 1, inplace = True)
best_shooter_df.sort_values(by = ['AVG SHOT PROB'], ascending = False).head(10)



,AVG SHOT PROB,NAME
77,0.590481,DeAndre Jordan
168,0.569746,Steven Adams
16,0.566666,Dwight Howard
119,0.566125,Tristan Thompson
130,0.550669,Kenneth Faried
120,0.546609,Jonas Valanciunas
128,0.545131,Enes Kanter
39,0.544481,Marcin Gortat
113,0.543641,Hassan Whiteside
144,0.540607,Andre Drummond


Testing

In [18]:
#Data prep
shot_df_original = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df_original = shot_df_original.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df_original = shot_df_original[shot_df_original['SEASON'] != "GAME_ID"]

#convert columns to numbers
shot_df_original[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]] = shot_df_original[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]].apply(pd.to_numeric)


points_df = pd.merge(X_test, shot_df_original, how = 'left', left_index=True, right_index=True)

points_df = points_df[['SHOT_DIST_x', 'DRIBBLES_x', 'CLOSE_DEF_DIST_x', 'TOUCH_TIME_x', 'SV_PLAYER_ID_x', 'CLOSE_DEF_SV_PLAYER_ID_x', 'SHOT_PROBABILITY', 'PT_VALUE', 'PTS']]

points_df.columns = ['SHOT_DIST', 'DRIBBLES', 'CLOSE_DEF_DIST', 'TOUCH_TIME', 'SV_PLAYER_ID', 'CLOSE_DEF_SV_PLAYER_ID', 'SHOT_PROBABILITY', 'PT_VALUE', 'PTS']

points_df['EXPECTED_POINTS'] = points_df['SHOT_PROBABILITY']*points_df['PT_VALUE']


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
Total = points_df['EXPECTED_POINTS'].sum()
print (Total)

66617.0582


In [20]:
Total = points_df['PTS'].sum()
print (Total)

67033.0


Log Loss Score

In [36]:
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss


In [37]:
# predict probabilities
probs = logmodel.predict_proba(X_test.drop(['SHOT_PROBABILITY'], axis = 1))
# keep the predictions for class 1 only
                               
probs = probs[:, 1]
# calculate log loss
loss = log_loss(y_test, probs)
loss

0.6615729943862341

In [38]:
auc_loss = roc_auc_score(y_test, probs)
auc_loss

0.6299726397388031

In [39]:
brier_loss = brier_score_loss(y_test, probs)
brier_loss

0.2345677743418934

In [25]:
from sklearn.metrics import r2_score


In [27]:
r2_score(y_test, probs)

0.051698573751231214

array([0.40203049, 0.56416794, 0.59280644, ..., 0.75992116, 0.36348273,
       0.48234962])